In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import importlib
import torch
import pandas as pd
import os
import sys
import json
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from recover.utils.utils import get_tensor_dataset
import reservoir as rdl
from copy import deepcopy
import numpy as np
from tqdm import tqdm

In [3]:
#####################################
# Methods to get different types of loaders
#####################################


def get_regular_valid_loader(trainer):
    return trainer.valid_loader

def get_test_loader(trainer):
    test_dataset = get_tensor_dataset(trainer.data, trainer.test_idxs)
    test_loader = DataLoader(test_dataset, batch_size=128)
    
    return test_loader


#####################################
# Iterator over trainers for a given config
#####################################


def trainer_iterator(config_file, path_to_conf):
    configuration = importlib.import_module("recover.config." + config_file).configuration

    # Loop over all runs for this configuration
    for run_dir in tqdm(os.listdir(os.path.join(path_to_conf, config_file))):
        print(run_dir)
        if run_dir.startswith('BasicTrainer'):

            this_run_results = {}

            # Load params for that run
            with open(os.path.join(path_to_conf, config_file, run_dir, 'params.json')) as f:
                params = json.load(f)

            # Load configuration (can contain grid_search args)
            this_run_config = deepcopy(configuration)

            # Replace grid_search args by the actual parameter for that run
            for key in this_run_config['trainer_config']:
                if type(this_run_config['trainer_config'][key]) is dict \
                and 'grid_search' in this_run_config['trainer_config'][key].keys():
                    
                    # If grid search over python classes, we need to load them
                    if type(params[key]) is str and params[key].startswith('<class'):
                        class_to_load = params[key]
                        class_to_load = class_to_load.split("'")[1]
                        class_to_load = class_to_load.rpartition('.')
                        path_to_class = class_to_load[0]
                        class_to_load_name = class_to_load[-1]
                        params[key] = getattr(importlib.import_module(path_to_class), 
                                              class_to_load_name)
                    this_run_config['trainer_config'][key] = params[key]
                    this_run_results[key] = params[key]

            # Load trainer
            trainer = this_run_config["trainer"](this_run_config["trainer_config"])

            # Find the checkpoint corresponding to the best epoch (always two checkpoints, 
            # corresponding to best and last epochs)
            cpt = 0
            checkpoint = None
            for dir_check in os.listdir(os.path.join(path_to_conf, config_file, run_dir)):
                if dir_check.startswith('checkpoint'):
                    cpt += 1
                    if checkpoint is None:
                        checkpoint = dir_check
                    else:
                        if int(dir_check.split('_')[-1]) < int(checkpoint.split('_')[-1]):
                            checkpoint = dir_check
                            
            if cpt == 2:
                # Only yield trainer if 2 checkpoints have been saved (corresponding to best and last epochs)

                # Load model
                trainer.model.load_state_dict(torch.load(path_to_conf + config_file + "/" + 
                                                 run_dir + "/" + checkpoint + "/model.pth",
                                                 map_location=torch.device('cpu')))
                print("Loaded model from", run_dir, checkpoint)
                
                yield trainer

#####################################
# Main evaluation method
#####################################


def evaluate_config(config_file, path_to_conf, get_eval_loader=get_regular_valid_loader):
    all_results = pd.DataFrame()

    for trainer in trainer_iterator(config_file, path_to_conf):
        
        this_run_results = {}

        # Evaluate
        eval_metrics, _ = trainer.eval_epoch(trainer.data, get_eval_loader(trainer), 
                                             trainer.model)

        # Create dataframe for this run
        print("this run results", this_run_results)
        print("eval metrics", eval_metrics)

        this_run_results = {**this_run_results, **eval_metrics}
        for key in this_run_results.keys():
            this_run_results[key] = [this_run_results[key]]

        this_run_df = pd.DataFrame.from_dict(this_run_results)

        all_results = all_results.append(this_run_df)

    all_results.reset_index()
        
    return all_results
        

## Pair level split (default)

In [3]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_evaluation_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.88it/s]

R2 0.144 \pm 0.025
spearman 0.454 \pm 0.008


In [4]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_evaluation_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.50it/s]

R2 0.329 \pm 0.034
spearman 0.453 \pm 0.011


## Drug Level Split

In [5]:
from recover.datasets.drugcomb_matrix_data import DrugCombMatrixDrugLevelSplitTest

def get_drug_split_test_loader(trainer):
    
    dl_split_data = DrugCombMatrixDrugLevelSplitTest(cell_line='MCF7',
                                     fp_bits=1024,
                                     fp_radius=2)
    dl_split_data.data.ddi_edge_response = dl_split_data.data.ddi_edge_bliss_max
    
    test_idxs = range(len(dl_split_data.data.ddi_edge_response))
    
    test_dataset = get_tensor_dataset(dl_split_data.data, test_idxs)
    test_loader = DataLoader(test_dataset, batch_size=128)
    
    return test_loader

In [6]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_drug_level_split_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_drug_split_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.28it/s]

R2 0.059 \pm 0.003
spearman 0.176 \pm 0.003


In [7]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_drug_level_split_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout


print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.13it/s]

R2 0.356 \pm 0.111
spearman 0.401 \pm 0.032


## Multi Cell Line

In [8]:
def get_mcf7_test_loader(trainer):

    mcf7_idxs = np.where(np.array(trainer.data.ddi_edge_classes) == trainer.data.cell_line_to_idx_dict['MCF7'])[0]
    
    mcf7_test_idxs = list(set(np.array(trainer.test_idxs)).intersection(mcf7_idxs))
    mcf7_test_idxs = torch.Tensor(mcf7_test_idxs).long()

    test_dataset = get_tensor_dataset(trainer.data, mcf7_test_idxs)
    test_loader = DataLoader(test_dataset, batch_size=128)
    
    return test_loader

In [9]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_evaluation_multi_cell_line_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_mcf7_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.40s/it]

R2 0.305 \pm 0.012
spearman 0.43 \pm 0.005


In [10]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="model_evaluation_multi_cell_line_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [12:10<00:00, 146.08s/it]

R2 0.393 \pm 0.015
spearman 0.52 \pm 0.006


## Cell Line Transfer

In [11]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="cell_line_transfer_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:26<00:00, 17.28s/it]

R2 0.399 \pm 0.01
spearman 0.389 \pm 0.036


In [12]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="cell_line_transfer_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [11:58<00:00, 143.76s/it]

R2 0.366 \pm 0.011
spearman 0.35 \pm 0.045


## Study Transfer

In [13]:
from recover.datasets.drugcomb_matrix_data import DrugCombMatrixTestAlmanac

def get_trimmed_Almanac_test_loader(trainer):
    
    dl_split_data = DrugCombMatrixTestAlmanac(cell_line=None,
                                     fp_bits=1024,
                                     fp_radius=2)
    dl_split_data.data.ddi_edge_response = dl_split_data.data.ddi_edge_bliss_max
    
    test_idxs = range(len(dl_split_data.data.ddi_edge_response))
    
    test_dataset = get_tensor_dataset(dl_split_data.data, test_idxs)
    test_loader = DataLoader(test_dataset, batch_size=128)
    
    return test_loader

In [14]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="pretrain_ONEIL_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_trimmed_Almanac_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.31s/it]

R2 0.004 \pm 0.002
spearman 0.096 \pm 0.026


In [15]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="pretrain_ONEIL_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_50179/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:54<00:00, 10.96s/it]

R2 0.378 \pm 0.021
spearman 0.64 \pm 0.009


## One hidden drug

In [4]:
from recover.datasets.drugcomb_matrix_data import DrugCombMatrixOneHiddenDrugSplitTest

def get_one_hidden_drug_split_test_loader(trainer):
    
    ohd_split_data = DrugCombMatrixOneHiddenDrugSplitTest(cell_line='MCF7',
                                     fp_bits=1024,
                                     fp_radius=2)
    ohd_split_data.data.ddi_edge_response = ohd_split_data.data.ddi_edge_bliss_max
    
    test_idxs = range(len(ohd_split_data.data.ddi_edge_response))
    
    test_dataset = get_tensor_dataset(ohd_split_data.data, test_idxs)
    test_loader = DataLoader(test_dataset, batch_size=128)
    
    return test_loader

In [5]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="one_hidden_drug_split_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_one_hidden_drug_split_test_loader)

sys.stdout = old_stdout # reset old stdout

print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_67729/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.68s/it]

R2 0.17 \pm 0.012
spearman 0.302 \pm 0.018


In [6]:
old_stdout = sys.stdout # backup current stdout
sys.stdout = open(os.devnull, "w")

all_results = evaluate_config(config_file="one_hidden_drug_split_deep_syn", 
             path_to_conf="/Users/paul/PycharmProjects/NewReservoir/"
                              "Recover/RayLogs/",
             get_eval_loader=get_regular_valid_loader)

sys.stdout = old_stdout # reset old stdout


print("R2", round(all_results['comb_r_squared'].mean(), 3), "\pm", 
      round(all_results['comb_r_squared'].std(),3))
print('spearman', round(all_results['spearman'].mean(), 3), "\pm", 
      round(all_results['spearman'].std(), 3))

  0%|                                                                                                                                                 | 0/5 [00:00<?, ?it/s]/var/folders/k3/yv9hq08j1vvbkzfmhdr7b05w0000gn/T/ipykernel_67729/1466826850.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_results = all_results.append(this_run_df)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.15it/s]

R2 0.283 \pm 0.079
spearman 0.424 \pm 0.056
